In [1]:
from pandas import read_csv
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB, BernoulliNB
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_travel = pd.read_csv('./shinkansen_travel_data/travel_data_train.csv')
train_survey = pd.read_csv('./shinkansen_travel_data/survey_data_train.csv')

In [3]:
def clean_data(data):
    # replace NaN with mean/mode for numerical and categoriacal
    for col in data.select_dtypes(include=['number']).columns:
        data[col].fillna(data[col].mean(), inplace=True)
    for col in data.select_dtypes(include=['object']).columns:
        data[col].fillna(data[col].mode()[0], inplace=True)
    # create binary columns for each categorical option
    data = pd.get_dummies(
        data=data, 
        columns=list(data.select_dtypes(include=['object']).columns)).astype(int)
    data.drop(['ID'],inplace=True, axis=1)
    return data

def concatenate(df1,df2):
    return pd.concat([df1, df2], axis=1)

def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

def drop_highly_correlated(data, threshold=0.7):
    correlation_matrix = data.corr().abs()
    upper_triangle = correlation_matrix.where(
        np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
    )
    highly_correlated_columns = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]
    data = data.drop(columns=highly_correlated_columns)
    return data

def correlation_matrix_plot(data):
    correlation_matrix = data.corr()
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        correlation_matrix, 
        mask=mask, 
        annot=True,
        cmap="coolwarm",
        vmax=1, 
        vmin=-1, 
        center=0,
        square=True,
        linewidths=0.5,
        cbar_kws={"shrink": 0.75}
    )
    plt.title("Correlation Matrix (Lower Triangle)")
    plt.show()

def logistic_regression(X_train, y_train, X_test):
    # add constant (i.e beta_0 intercept term)
    X_train = sm.add_constant(X_train)
    X_test = sm.add_constant(X_test)
    model = sm.Logit(y_train, X_train)
    result = model.fit()
    # print the summary
    print(result.summary())
    # predict
    predictions = result.predict(X_test)
    # convert probabilities to binary predictions (0/1)
    binary_predictions = (predictions >= 0.5).astype(int)
    return binary_predictions, model


In [4]:
train_travel, train_survey = clean_data(train_travel), clean_data(train_survey)
train_data = concatenate(train_travel, train_survey)

In [5]:
#correlation_matrix_plot(train_travel)

In [6]:
#train_travel.info()

In [7]:
print(train_data.shape)
columns_to_remove_outliers = [
    'Travel_Distance', 
    'DepartureDelay_in_Mins',
    'ArrivalDelay_in_Mins', 
    'Onboard_service_extremely poor', 
    'Onlinebooking_Ease_extremely poor', 
    'Platform_location_very inconvinient'
]
for column in columns_to_remove_outliers:
    train_data = remove_outliers_iqr(train_data, column)

(94379, 98)


In [8]:
assert train_data.isnull().sum().sum() == 0

In [9]:
print(train_data.shape)

(68605, 98)


In [10]:
y = train_data['Overall_Experience']
X = train_data.drop(['Overall_Experience'], axis=1)
# drop highly correlated columns to reduce feature space
X = drop_highly_correlated(X, threshold=0.5)
print(X.shape)

(68605, 66)


In [11]:
rank = np.linalg.matrix_rank(X)
print(f"Rank of the design matrix: {rank}")
print(f"Number of features: {X.shape[1]}")
# checking for duplicate columns
duplicate_columns = X.columns[X.columns.duplicated()]
print(f"Duplicate Columns: {duplicate_columns.tolist()}")
# checking for constant columns
constant_columns = [col for col in X.columns if X[col].nunique() <= 1]
print(f"Constant Columns: {constant_columns}")
X = X.drop(columns=constant_columns)
print(X.shape)

# calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
# drop features with high VIF
high_vif_features = vif_data[vif_data["VIF"] > 10]["Feature"].tolist()
X = X.drop(columns=high_vif_features)
# check the updated shape and rank
print(f"Remaining columns after dropping high VIF features: {X.shape[1]}")
rank = np.linalg.matrix_rank(X)
print(f"Rank of the updated design matrix: {rank}")

Rank of the design matrix: 55
Number of features: 66
Duplicate Columns: []
Constant Columns: ['Platform_location_very inconvinient', 'Online_support_extremely poor', 'Onlinebooking_Ease_extremely poor', 'Onboard_service_extremely poor', 'Checkin_service_extremely poor', 'Cleanliness_extremely poor', 'Online_boarding_extremely poor']
(68605, 59)
Remaining columns after dropping high VIF features: 30
Rank of the updated design matrix: 30


In [12]:
# split data into train and test samples
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# call log reg function from statsmodel
y_preds, model = logistic_regression(X_train, y_train, X_test)

Optimization terminated successfully.
         Current function value: 0.356659
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:     Overall_Experience   No. Observations:                54884
Model:                          Logit   Df Residuals:                    54853
Method:                           MLE   Df Model:                           30
Date:                Fri, 01 Nov 2024   Pseudo R-squ.:                  0.4778
Time:                        17:06:52   Log-Likelihood:                -19575.
converged:                       True   LL-Null:                       -37483.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const                                -2.4603      0.081    -30

In [13]:
# eval
accuracy = accuracy_score(y_test, y_preds)
print(f'Accuracy: {accuracy:.4f}')
# classification report: will make this a plot
print("Classification Report:")
print(classification_report(y_test, y_preds))

Accuracy: 0.8449
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.83      0.82      5856
           1       0.87      0.86      0.86      7865

    accuracy                           0.84     13721
   macro avg       0.84      0.84      0.84     13721
weighted avg       0.85      0.84      0.85     13721



In [14]:
# fit Bernoulli Naive Bayes model
bnb = BernoulliNB()
bnb.fit(X_train, y_train)
# pred and eval
y_pred = bnb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Bernoulli Naive Bayes Accuracy: {accuracy:.2f}')

# Gausiian NB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
# predict and evaluate
y_pred = gnb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Gaussian Naive Bayes Accuracy: {accuracy:.2f}')

Bernoulli Naive Bayes Accuracy: 0.82
Gaussian Naive Bayes Accuracy: 0.81
